In [1]:
import pandas as pd
pd.options.display.max_columns = None

import geopandas as gpd
from sqlalchemy import create_engine, types
import psycopg2
import numpy as np
from getpass import getpass
from shapely.geometry import Point
from geoalchemy2.types import Geometry, WKTElement
import re

In [2]:
ls '/data/user/Data/2015 Data/Final_data_2015_DwC'


AMBON2015_zooplankton_Abundance_150_DC.csv  AMBON_station_masterlist.csv
AMBON2015_zooplankton_biomass_150_DC.csv


In [20]:
df = pd.read_csv('/data/user/Data/2015 Data/Final_data_2015_DwC/AMBON2015_zooplankton_Abundance_150_DC.csv', parse_dates=['eventDate'])

df.columns = df.columns.str.lower()

#df.dtypes

In [21]:
df.drop(columns = ['unnamed: 28', 'unnamed: 29', 'unnamed: 30', 'unnamed: 31', 'unnamed: 32', 'unnamed: 33', 'unnamed: 34', 'unnamed: 35', 'unnamed: 36', 'acartia longiremis'], inplace=True)

# replace unaccepted aphia id with accepted
df.taxonid.replace(322599, 101800, inplace=True)


# convert to gpd, add geom column
df = gpd.GeoDataFrame(df)
df['geom'] = df.apply(lambda r: WKTElement(Point(r.decimallongitude, r.decimallatitude).wkt, srid=4326), axis=1)


In [22]:
DB_PASSWORD = getpass('Enter spatial db password:')
db_url='postgresql://db1.axiomptk:5432/spatial?user=spatial&password='+DB_PASSWORD
engine = create_engine(db_url)

df.to_sql('ambon_2015_zooplankton_abundance', 
                engine, 
                schema='ambon_zooplankton', 
                chunksize=500, 
                if_exists='replace', 
                dtype={'geom': Geometry('POINT', srid=4326)}
               )

Enter spatial db password:········


In [16]:
df_bio = pd.read_csv('/data/user/Data/2015 Data/Final_data_2015_DwC/AMBON2015_zooplankton_biomass_150_DC.csv', parse_dates=['eventDate'])

df_bio.columns = df_bio.columns.str.lower()

# df_bio.dtypes

In [17]:
df_bio.drop(columns = ['unnamed: 28', 'unnamed: 29', 'unnamed: 30', 'unnamed: 31', 'unnamed: 32', 'unnamed: 33', 'unnamed: 34', 'unnamed: 35', 'unnamed: 36', 'acartia longiremis'], inplace=True)

df_bio.rename(columns={'biomass (mg dw/m3)': 'biomass_mg_dry_m3'}, inplace=True)

# replace unaccepted aphia id with accepted
df_bio.taxonid.replace(322599, 101800, inplace=True)

# convert to gpd, add geom column
df_bio = gpd.GeoDataFrame(df_bio)
df_bio['geom'] = df_bio.apply(lambda r: WKTElement(Point(r.decimallongitude, r.decimallatitude).wkt, srid=4326), axis=1)

df_bio.head()

,eventdate,decimallatitude,decimallongitude,eventid,parenteventid,basisofrecord,minimumdepthinmeters,maximumdepthinmeters,samplngprotocol,scientificname,nameaccordingtoid,nameaccordingto,taxonid,kingdom,phylum,subphylum,class,subclass,infraclass,order,suborder,infraorder,family,genus,species,life stage comment,organismquantitytype,biomass_mg_dry_m3,geom
0,2015-08-11 21:20:00,67.67231,-168.9545,DBO3.8,AMBON,PreservedSpecimen,0,42,Twin-ring; 60 cm diameter; mesh 150,Acartia spp.,https://dx.doi.org/10.14284/170,WoRMS,104108,Animalia,Arthropoda,Crustacea,Maxillopoda,Copepoda,Neocopepoda,Calanoida,NaN,NaN,Acartiidae,Acartia,NaN,NaN,mgdryweight/m3,0.0582,POINT (-168.9545 67.67231)
1,2015-08-11 21:20:00,67.67231,-168.9545,DBO3.8,AMBON,PreservedSpecimen,0,42,Twin-ring; 60 cm diameter; mesh 150,Aglantha digitale,https://dx.doi.org/10.14284/170,WoRMS,117849,Animalia,Cnidaria,NaN,Hydrozoa,Trachylina,NaN,Narcomedusae,Narcomedusida,NaN,Rhopalonematidae,Aglantha,digitale,NaN,mgdryweight/m3,1.8847,POINT (-168.9545 67.67231)
2,2015-08-11 21:20:00,67.67231,-168.9545,DBO3.8,AMBON,PreservedSpecimen,0,42,Twin-ring; 60 cm diameter; mesh 150,Bivalvia,https://dx.doi.org/10.14284/170,WoRMS,105,Animalia,Mollusca,NaN,Bivalvia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,larvae,mgdryweight/m3,60.3302,POINT (-168.9545 67.67231)
3,2015-08-11 21:20:00,67.67231,-168.9545,DBO3.8,AMBON,PreservedSpecimen,0,42,Twin-ring; 60 cm diameter; mesh 150,Brachyura,https://dx.doi.org/10.14284/170,WoRMS,106673,Animalia,Arthropoda,Crustacea,Malacostraca,Eumalacostraca,NaN,Decapoda,Pleocyemata,Brachyura,NaN,NaN,NaN,zoea,mgdryweight/m3,0.2442,POINT (-168.9545 67.67231)
4,2015-08-11 21:20:00,67.67231,-168.9545,DBO3.8,AMBON,PreservedSpecimen,0,42,Twin-ring; 60 cm diameter; mesh 150,Calanoida,https://dx.doi.org/10.14284/170,WoRMS,1100,Animalia,Arthropoda,Crustacea,Maxillopoda,Copepoda,Neocopepoda,Calanoida,NaN,NaN,NaN,NaN,NaN,nauplii,mgdryweight/m3,0.5526,POINT (-168.9545 67.67231)


In [23]:
DB_PASSWORD = getpass('Enter spatial db password:')
db_url='postgresql://db1.axiomptk:5432/spatial?user=spatial&password='+DB_PASSWORD
engine = create_engine(db_url)

df_bio.to_sql('ambon_2015_zooplankton_biomass', 
                engine, 
                schema='ambon_zooplankton', 
                chunksize=500, 
                if_exists='replace', 
                dtype={'geom': Geometry('POINT', srid=4326)}
               )

Enter spatial db password:········
